# Download Dataset GTZAN da Kaggle

Questo notebook scarica automaticamente il dataset GTZAN da Kaggle e lo organizza nella struttura del progetto.

## Prerequisiti

✅ Credenziali Kaggle configurate (file `kaggle/kaggle.json` nel progetto)  
✅ Librerie `kaggle` e `kagglehub` installate  
✅ Ambiente virtuale attivo  

**Nota:** Se non hai ancora configurato Kaggle, esegui: `python scripts/setup_kaggle.py`

In [5]:
import os
import sys
import shutil
from pathlib import Path

# Setup ambiente Kaggle
project_root = Path.cwd().parent
kaggle_dir = project_root / 'kaggle'

print("🎵 GTZAN Dataset Download")
print("=" * 30)
print(f"📁 Progetto: {project_root.name}")
print(f"🔑 Credenziali: {kaggle_dir}")

# Verifica credenziali
kaggle_config_path = kaggle_dir / 'kaggle.json'
if not kaggle_config_path.exists():
    print("❌ Credenziali mancanti - esegui: python scripts/setup_kaggle.py")
    sys.exit(1)

print("✅ Credenziali Kaggle trovate")

# Configurazione manuale per evitare auto-autenticazione
os.environ['KAGGLE_CONFIG_DIR'] = str(kaggle_dir)
os.environ['KAGGLE_USERNAME'] = ''  # Reset per forzare lettura da file
os.environ['KAGGLE_KEY'] = ''       # Reset per forzare lettura da file

# Import delle librerie Kaggle
try:
    from kaggle.api.kaggle_api_extended import KaggleApi
    import kagglehub
    
    # Crea istanza API manuale (senza auto-autenticazione)
    kaggle_api = KaggleApi()
    kaggle_api.config_dir = str(kaggle_dir)
    kaggle_api.config_file = str(kaggle_config_path)
    
    print("✅ Librerie Kaggle caricate")
    
except Exception as e:
    print(f"❌ Errore import: {e}")
    sys.exit(1)

🎵 GTZAN Dataset Download
📁 Progetto: naml_project
🔑 Credenziali: /home/alepot55/Desktop/projects/naml_project/kaggle
✅ Credenziali Kaggle trovate
✅ Librerie Kaggle caricate


## Download Dataset GTZAN

In [6]:
# Dataset GTZAN su Kaggle
dataset_name = "andradaolteanu/gtzan-dataset-music-genre-classification"

print(f"📥 Downloading: {dataset_name}")
print("⏳ Questo richiederà alcuni minuti...")

try:
    # Test autenticazione con la nostra API
    print("🔐 Test autenticazione...")
    kaggle_api.authenticate()
    print("✅ Autenticazione Kaggle OK")
    
    # Download del dataset usando kagglehub
    print("📥 Avvio download con kagglehub...")
    print("💡 Nota: Se già presente in cache, sarà veloce")
    
    path = kagglehub.dataset_download(dataset_name)
    print(f"✅ Dataset disponibile in: {path}")
    
    # Verifica se è una cartella valida
    path_obj = Path(path)
    if not path_obj.exists():
        raise FileNotFoundError(f"Percorso non trovato: {path}")
    
    print(f"📂 Percorso verificato: {path_obj}")
    print(f"📊 È una directory: {path_obj.is_dir()}")
    
    # Conta file totali (con timeout per evitare blocchi)
    print("🔍 Analisi contenuto...")
    downloaded_files = list(path_obj.rglob('*'))
    print(f"📁 File totali trovati: {len(downloaded_files)}")
    
    # Struttura principale (solo primo livello)
    print("\n📂 Struttura primo livello:")
    items = list(path_obj.iterdir())
    print(f"   📋 Elementi al primo livello: {len(items)}")
    
    for item in items:
        if item.is_dir():
            # Conta veloce senza ricorsione profonda
            try:
                first_level_count = len(list(item.iterdir()))
                print(f"  📂 {item.name}: {first_level_count} elementi")
            except:
                print(f"  📂 {item.name}: (non accessibile)")
        else:
            print(f"  📄 {item.name} ({item.stat().st_size / 1024:.1f} KB)")
            
    print(f"\n🎯 Download completato! Percorso: {path}")
            
except Exception as e:
    print(f"❌ Errore: {e}")
    print(f"❌ Tipo errore: {type(e)}")
    print("\n💡 Soluzioni:")
    print("   - Verifica che kaggle.json sia in naml_project/kaggle/")
    print("   - Verifica connessione internet")
    print("   - Rigenera token su kaggle.com/account")
    print("   - Riavvia il kernel e riprova")
    raise

📥 Downloading: andradaolteanu/gtzan-dataset-music-genre-classification
⏳ Questo richiederà alcuni minuti...
🔐 Test autenticazione...
✅ Autenticazione Kaggle OK
📥 Avvio download con kagglehub...
💡 Nota: Se già presente in cache, sarà veloce
✅ Dataset disponibile in: /home/alepot55/.cache/kagglehub/datasets/andradaolteanu/gtzan-dataset-music-genre-classification/versions/1
📂 Percorso verificato: /home/alepot55/.cache/kagglehub/datasets/andradaolteanu/gtzan-dataset-music-genre-classification/versions/1
📊 È una directory: True
🔍 Analisi contenuto...
📁 File totali trovati: 2024

📂 Struttura primo livello:
   📋 Elementi al primo livello: 1
  📂 Data: 4 elementi

🎯 Download completato! Percorso: /home/alepot55/.cache/kagglehub/datasets/andradaolteanu/gtzan-dataset-music-genre-classification/versions/1


## Organizzazione Dataset nel Progetto

In [7]:
# Organizza il dataset nella struttura del progetto
data_dir = project_root / 'data' / 'gtzan'
data_dir.mkdir(parents=True, exist_ok=True)

print(f"📁 Organizzazione in: {data_dir}")

if 'path' in locals():
    source_path = Path(path)
    print(f"📂 Sorgente: {source_path}")
    
    # Verifica rapida che la sorgente esista
    if not source_path.exists():
        print(f"❌ Percorso sorgente non esiste: {source_path}")
    else:
        print(f"✅ Percorso sorgente verificato")
    
    # Componenti da trovare
    components_found = {}
    
    print(f"\n🔍 Ricerca componenti...")
    
    # 1. Cerca cartella genres (ricerca ottimizzata - max 3 livelli)
    print("  🎵 Cercando cartella genres...")
    for subdir in source_path.rglob('*'):
        # Limita profondità per evitare ricerche infinite
        if len(subdir.parts) - len(source_path.parts) > 3:
            continue
        if subdir.is_dir() and 'genres_original' in subdir.name:
            components_found['genres'] = subdir
            print(f"    ✅ Trovata: {subdir.relative_to(source_path)}")
            break
    else:
        print("    ❌ Cartella genres non trovata")
    
    # 2. Cerca cartella images (ricerca ottimizzata)
    print("  🖼️  Cercando cartella images...")
    for subdir in source_path.rglob('*'):
        if len(subdir.parts) - len(source_path.parts) > 3:
            continue
        if subdir.is_dir() and 'images' in subdir.name.lower():
            components_found['images'] = subdir
            print(f"    ✅ Trovata: {subdir.relative_to(source_path)}")
            break
    else:
        print("    ❌ Cartella images non trovata")
    
    # 3. Cerca file CSV (solo primi 2 livelli)
    print("  📊 Cercando file CSV...")
    csv_files = []
    for level in [source_path, *source_path.iterdir()]:
        if level.is_dir():
            csv_found = list(level.glob('*.csv'))
            csv_files.extend(csv_found)
    
    if csv_files:
        components_found['csv'] = csv_files
        print(f"    ✅ Trovati {len(csv_files)} file CSV")
    else:
        print("    ❌ File CSV non trovati")
    
    print(f"\n📋 Riepilogo componenti trovati:")
    for comp, found_path in components_found.items():
        if comp == 'csv':
            print(f"  📊 CSV: {len(found_path)} file")
        else:
            print(f"  📂 {comp}: {found_path.name}")
    
    # Copia i componenti con progress
    print(f"\n📋 Inizio copia componenti...")
    
    # Copia cartella genres
    if 'genres' in components_found:
        target_genres = data_dir / 'genres'
        if not target_genres.exists():
            print(f"  🎵 Copiando cartella genres...")
            print(f"      Da: {components_found['genres']}")
            print(f"      A:  {target_genres}")
            shutil.copytree(components_found['genres'], target_genres)
            print(f"      ✅ Cartella genres copiata")
        else:
            print(f"  ⏭️  Cartella genres già presente")
    
    # Copia cartella images
    if 'images' in components_found:
        target_images = data_dir / 'images'
        if not target_images.exists():
            print(f"  🖼️  Copiando cartella images...")
            print(f"      Da: {components_found['images']}")
            print(f"      A:  {target_images}")
            shutil.copytree(components_found['images'], target_images)
            print(f"      ✅ Cartella images copiata")
        else:
            print(f"  ⏭️  Cartella images già presente")
    
    # Copia file CSV
    if 'csv' in components_found:
        print(f"  📊 Copiando {len(components_found['csv'])} file CSV...")
        for i, csv_file in enumerate(components_found['csv'], 1):
            target_csv = data_dir / csv_file.name
            if not target_csv.exists():
                shutil.copy2(csv_file, target_csv)
                print(f"      ✅ {i}/{len(components_found['csv'])}: {csv_file.name}")
            else:
                print(f"      ⏭️  {i}/{len(components_found['csv'])}: {csv_file.name} già presente")
    
    print("\n🎉 Organizzazione completata!")
    
    # Riepilogo veloce finale
    print("\n📊 Verifica rapida dataset:")
    
    # Conta generi (solo primo livello)
    genres_dir = data_dir / 'genres'
    if genres_dir.exists():
        genre_dirs = [d for d in genres_dir.iterdir() if d.is_dir()]
        print(f"  🎵 Generi: {len(genre_dirs)} cartelle")
        # Conta solo primo genere per velocità
        if genre_dirs:
            sample_count = len(list(genre_dirs[0].glob('*.wav')))
            print(f"      Esempio {genre_dirs[0].name}: {sample_count} file")
    
    # Conta immagini
    images_dir = data_dir / 'images'
    if images_dir.exists():
        image_count = len(list(images_dir.glob('*.png')))
        print(f"  🖼️  Immagini: {image_count} file")
    
    # Lista CSV
    csv_files_final = list(data_dir.glob('*.csv'))
    if csv_files_final:
        print(f"  📊 CSV: {[f.name for f in csv_files_final]}")
        
else:
    print("❌ Variabile 'path' non trovata - esegui prima il download")

📁 Organizzazione in: /home/alepot55/Desktop/projects/naml_project/data/gtzan
📂 Sorgente: /home/alepot55/.cache/kagglehub/datasets/andradaolteanu/gtzan-dataset-music-genre-classification/versions/1
✅ Percorso sorgente verificato

🔍 Ricerca componenti...
  🎵 Cercando cartella genres...
    ✅ Trovata: Data/genres_original
  🖼️  Cercando cartella images...
    ✅ Trovata: Data/images_original
  📊 Cercando file CSV...
    ✅ Trovati 2 file CSV

📋 Riepilogo componenti trovati:
  📂 genres: genres_original
  📂 images: images_original
  📊 CSV: 2 file

📋 Inizio copia componenti...
  ⏭️  Cartella genres già presente
  ⏭️  Cartella images già presente
  📊 Copiando 2 file CSV...
      ⏭️  1/2: features_30_sec.csv già presente
      ⏭️  2/2: features_3_sec.csv già presente

🎉 Organizzazione completata!

📊 Verifica rapida dataset:
  🎵 Generi: 10 cartelle
      Esempio pop: 100 file
  🖼️  Immagini: 0 file
  📊 CSV: ['features_30_sec.csv', 'features_3_sec.csv']


## Verifica Dataset Finale

In [8]:
# Verifica finale del dataset completo
data_dir = project_root / 'data' / 'gtzan'

if data_dir.exists():
    print("🎵 DATASET GTZAN COMPLETO PRONTO!")
    print(f"📁 Percorso: {data_dir}")
    print("=" * 40)
    
    # Verifica componenti
    components_status = {}
    
    # 1. Verifica cartella genres
    genres_dir = data_dir / 'genres'
    if genres_dir.exists():
        genres = [d for d in genres_dir.iterdir() if d.is_dir()]
        total_audio = sum(len(list(g.glob('*.wav'))) for g in genres)
        components_status['genres'] = f"{len(genres)} generi, {total_audio} file audio"
        print(f"✅ Genres: {components_status['genres']}")
    else:
        print("❌ Cartella genres mancante")
    
    # 2. Verifica cartella images  
    images_dir = data_dir / 'images'
    if images_dir.exists():
        images = list(images_dir.glob('*.png'))
        components_status['images'] = f"{len(images)} spettrogrammi"
        print(f"✅ Images: {components_status['images']}")
    else:
        print("❌ Cartella images mancante")
    
    # 3. Verifica file CSV
    csv_files = list(data_dir.glob('*.csv'))
    if csv_files:
        components_status['csv'] = [f.name for f in csv_files]
        print(f"✅ CSV: {components_status['csv']}")
    else:
        print("❌ File CSV mancanti")
    
    # Riepilogo struttura
    print(f"\n📂 Struttura finale:")
    print(f"📁 data/gtzan/")
    if genres_dir.exists():
        print(f"  ├── genres/ ({len([d for d in genres_dir.iterdir() if d.is_dir()])} cartelle)")
    if images_dir.exists():
        print(f"  ├── images/ ({len(list(images_dir.glob('*.png')))} file)")
    for csv in csv_files:
        print(f"  └── {csv.name}")
    
    if len(components_status) >= 3:
        print(f"\n🎯 TUTTO PRONTO PER L'ANALISI! 🎵")
    else:
        print(f"\n⚠️  Alcuni componenti mancanti - ricontrolla il download")
        
else:
    print("❌ Dataset non trovato")
    print("💡 Esegui le celle precedenti per il download")

🎵 DATASET GTZAN COMPLETO PRONTO!
📁 Percorso: /home/alepot55/Desktop/projects/naml_project/data/gtzan
✅ Genres: 10 generi, 1000 file audio
✅ Images: 0 spettrogrammi
✅ CSV: ['features_30_sec.csv', 'features_3_sec.csv']

📂 Struttura finale:
📁 data/gtzan/
  ├── genres/ (10 cartelle)
  ├── images/ (0 file)
  └── features_30_sec.csv
  └── features_3_sec.csv

🎯 TUTTO PRONTO PER L'ANALISI! 🎵
